In [18]:
from Bio import SeqIO
from pathlib import Path
from Bio.SeqRecord import SeqRecord
import pandas as pd
import re
import os
import numpy as np

identifier = r"CYP[\d+][A-Z+][\d+]"
# pattern_family = re.compile(r"(?<=CYP)[\d]+")
pattern_id = re.compile(r"(?<=CYP)[\d]+[A-Z]+[\d]+")
pattern_fam = re.compile(r"^[\d]+")
pattern_subfam = re.compile(r"[A-Z]+")
pattern_gene = re.compile(r"[\d]+$")

test = "CYP44LK4"
# print(re.search(pattern_family, test).group())
print(re.search(pattern_id, test).group())
test = "44LK4"
print(re.search(pattern_fam, test).group())
print(re.search(pattern_subfam, test).group())
print(re.search(pattern_gene, test).group())


def match_or_none(pattern, s):
    try:
        out = re.search(pattern, s).group()
    except:
        out = None
    return out

44LK4
44
LK
4


In [2]:
data_dir = Path("/hits/fast/cme/bodynems/data")
seqs: list[SeqRecord] = list(
    SeqIO.parse(data_dir / "backup" / "arthropod.fasta", format="fasta")
)
print(len(seqs))

stats = []
for seq in seqs:
    stats.append([seq.id, len(seq)])

stats_df = pd.DataFrame(stats, columns=["id_full", "length"])

11118


In [3]:
stats_df["id"] = stats_df["id_full"].map(lambda s: match_or_none(pattern_id, s))
stats_df["family"] = stats_df["id"].map(lambda s: match_or_none(pattern_fam, s))
stats_df["subfamily"] = stats_df["id"].map(lambda s: match_or_none(pattern_subfam, s))
stats_df["gene"] = stats_df["id"].map(lambda s: match_or_none(pattern_gene, s))


families = stats_df["family"].value_counts()
families = families[families > 4].index

id_map = dict(zip([seq.id for seq in seqs], stats_df["family"]))

seq_groups = {}
for family in families:
    seq_groups[family] = list(filter(lambda seq: id_map[seq.id] == family, seqs))

out_dir = data_dir / "paper" / "arthropod"
# os.makedirs(out_dir, exist_ok=True)

In [ ]:
from tqdm import tqdm

for family, group in tqdm(seq_groups.items()):
    dataset_dir = out_dir / f"CYP_{family}"
    # os.makedirs(dataset_dir)
    # SeqIO.write(group, dataset_dir / "sequences.fasta", "fasta")

100%|██████████| 237/237 [00:00<00:00, 309.72it/s]


In [ ]:
import shutil

# Special treatment for the largest groups (too big)
group_lens = sorted(
    {key: len(group) for key, group in seq_groups.items()},
    key=lambda key: len(seq_groups[key]),
)[::-1]
large_groups = {key: len(seq_groups[key]) for key in group_lens[:3]}

print(large_groups)

MAX_SIZE = 200
np.random.seed(1)

for family, size in large_groups.items():
    start = 0
    sizes = (25 + np.random.rand(int(np.sqrt(size))) * 200).astype(int)
    n = len(seq_groups[family])
    for suffix, size in enumerate(sizes):
        if start + size > n - 25:
            size = n
        subgroup = seq_groups[family][start : start + size]
        dataset_dir = out_dir / f"CYP_{family}_{suffix}"

        # if os.path.exists(dataset_dir):
        #     shutil.rmtree(dataset_dir)
        # os.makedirs(dataset_dir)
        # print(dataset_dir, len(subgroup))
        # SeqIO.write(subgroup, dataset_dir / "sequences.fasta", "fasta")

        if size == n:
            break
        start += size

{'6': 1927, '4': 1755, '9': 758}
/hits/fast/cme/bodynems/data/paper/arthropod/CYP_6_0 108
/hits/fast/cme/bodynems/data/paper/arthropod/CYP_6_1 169
/hits/fast/cme/bodynems/data/paper/arthropod/CYP_6_2 25
/hits/fast/cme/bodynems/data/paper/arthropod/CYP_6_3 85
/hits/fast/cme/bodynems/data/paper/arthropod/CYP_6_4 54
/hits/fast/cme/bodynems/data/paper/arthropod/CYP_6_5 43
/hits/fast/cme/bodynems/data/paper/arthropod/CYP_6_6 62
/hits/fast/cme/bodynems/data/paper/arthropod/CYP_6_7 94
/hits/fast/cme/bodynems/data/paper/arthropod/CYP_6_8 104
/hits/fast/cme/bodynems/data/paper/arthropod/CYP_6_9 132
/hits/fast/cme/bodynems/data/paper/arthropod/CYP_6_10 108
/hits/fast/cme/bodynems/data/paper/arthropod/CYP_6_11 162
/hits/fast/cme/bodynems/data/paper/arthropod/CYP_6_12 65
/hits/fast/cme/bodynems/data/paper/arthropod/CYP_6_13 200
/hits/fast/cme/bodynems/data/paper/arthropod/CYP_6_14 30
/hits/fast/cme/bodynems/data/paper/arthropod/CYP_6_15 159
/hits/fast/cme/bodynems/data/paper/arthropod/CYP_6_16 108